In [ ]:
from google.colab import files
uploaded = files.upload()

import pandas as pd

df = pd.read_csv("Housing Data Sets.csv")
df.head()

df.columns = [
    "description",
    "bathrooms",
    "bedrooms",
    "car_spaces",
    "floor_area_sqm",
    "land_size_sqm",
    "location",
    "price",
    "category"
]

numeric_cols = [
    "bathrooms", "bedrooms", "car_spaces",
    "floor_area_sqm", "land_size_sqm", "price"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df.isnull().sum()

df["description"] = df["description"].fillna("")
df["location"] = df["location"].fillna("Unknown")
df["category"] = df["category"].fillna("Uncategorized")

df = df[
    (df["bathrooms"] >= 0) &
    (df["bedrooms"] >= 0) &
    (df["car_spaces"] >= 0) &
    (df["floor_area_sqm"] >= 1) &
    (df["land_size_sqm"] >= 1) &
    (df["price"] >= 1)
]

text_cols = ["description", "location", "category"]

for col in text_cols:
    df[col] = df[col].str.lower().str.strip()

import re

df["description"] = df["description"].apply(
    lambda x: re.sub(r"[^a-zA-Z0-9\s]", "", x)
)

df = df.drop_duplicates()

df.info()
df.describe()


Saving Housing Data Sets.csv to Housing Data Sets.csv
<class 'pandas.core.frame.DataFrame'>
Index: 5313 entries, 0 to 7493
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   description     5313 non-null   object 
 1   bathrooms       5313 non-null   float64
 2   bedrooms        5313 non-null   float64
 3   car_spaces      5313 non-null   int64  
 4   floor_area_sqm  5313 non-null   float64
 5   land_size_sqm   5313 non-null   float64
 6   location        5313 non-null   object 
 7   price           5313 non-null   float64
 8   category        5313 non-null   object 
dtypes: float64(5), int64(1), object(3)
memory usage: 415.1+ KB


,bathrooms,bedrooms,car_spaces,floor_area_sqm,land_size_sqm,price
count,5313.000000,5313.000000,5313.000000,5313.000000,5313.000000,5.313000e+03
mean,2.764069,3.241389,0.895351,281.656217,290.780442,3.462045e+07
std,1.924840,1.669776,1.813594,502.966029,971.150670,9.482551e+07
min,0.000000,0.000000,0.000000,18.000000,25.000000,4.000000e+04
25%,1.000000,2.000000,0.000000,72.000000,80.000000,4.100000e+06
50%,3.000000,3.000000,0.000000,142.000000,123.000000,8.137000e+06
75%,4.000000,4.000000,1.000000,300.000000,254.000000,2.500000e+07
max,43.000000,22.000000,50.000000,8746.000000,41321.000000,2.500000e+09


In [ ]:
from google.colab import files
uploaded = files.upload()

df["text"] = df["description"] + " " + df["location"]
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=5000
)

tfidf_matrix = tfidf.fit_transform(df["text"])

from sklearn.metrics.pairwise import cosine_similarity

def recommend_homes(user_input, top_n=5):
    user_vec = tfidf.transform([user_input])
    similarity = cosine_similarity(user_vec, tfidf_matrix)
    top_indices = similarity[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

def filter_homes(
    category=None,
    min_price=None,
    max_price=None,
    bedrooms=None
):
    result = df.copy()

    if category:
        result = result[result["category"] == category]

    if bedrooms:
        result = result[result["bedrooms"] >= bedrooms]

    if min_price:
        result = result[result["price"] >= min_price]

    if max_price:
        result = result[result["price"] <= max_price]

    return result

features = [
    "bedrooms",
    "bathrooms",
    "car_spaces",
    "floor_area_sqm",
    "land_size_sqm"
]


X = df[features]
y = df["price"]



from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)

model.fit(X_train, y_train)

base_price = model.predict(X_test[:1])[0]

def future_price(current_price, years, annual_growth=0.05):
    return current_price * ((1 + annual_growth) ** years)


Saving Housing Data Sets.csv to Housing Data Sets (1).csv


In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Load CSV
df = pd.read_csv("Housing Data Sets.csv")

# Print original columns to see what we're working with
print("Original columns:", df.columns.tolist())

# Rename columns to simpler names (adjust these based on your actual column names)
# Common variations: "Description", "Bathrooms", "Bedrooms", etc.
df.columns = [
    "description",
    "bathrooms",
    "bedrooms",
    "car_spaces",
    "floor_area_sqm",
    "land_size_sqm",
    "location",
    "price",
    "category"
]

# Fill missing values
df["description"] = df["description"].fillna("")
df["location"] = df["location"].fillna("")

# Create combined text field for recommendation
df["text"] = df["description"] + " " + df["location"]

# Display first few rows
print("\nData preview:")
print(df.head())

# Check for missing values in numeric columns
print("\nMissing values:")
print(df.isnull().sum())

# Fill missing numeric values with median
numeric_cols = ["bathrooms", "bedrooms", "car_spaces", "floor_area_sqm", "land_size_sqm", "price"]
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

# ===== RECOMMENDATION SYSTEM =====
print("\n=== Building Recommendation System ===")
tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=5000
)
tfidf_matrix = tfidf.fit_transform(df["text"])

def recommend_homes(user_input, top_n=5):
    """Recommend homes based on text description"""
    user_vec = tfidf.transform([user_input])
    similarity = cosine_similarity(user_vec, tfidf_matrix)
    top_indices = similarity[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

# Test recommendation
print("\nTesting recommendation:")
recommended = recommend_homes("3 bedroom house near school with parking", 5)
print(recommended[["description", "bedrooms", "bathrooms", "price", "location"]])

# ===== PRICE PREDICTION MODEL =====
print("\n=== Building Price Prediction Model ===")
features = [
    "bedrooms",
    "bathrooms",
    "car_spaces",
    "floor_area_sqm",
    "land_size_sqm"
]

X = df[features]
y = df["price"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train model
model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:")
print(f"MAE: {mae:,.2f}")
print(f"R²: {r2:.4f}")

# ===== FUTURE PRICE PREDICTION =====
def predict_future_price(house_features, years, annual_growth=0.05):
    """
    Predict future price of a house

    Parameters:
    - house_features: dict or Series with keys: bedrooms, bathrooms, car_spaces, floor_area_sqm, land_size_sqm
    - years: number of years in the future
    - annual_growth: expected annual growth rate (default 5%)

    Returns:
    - current_price: predicted current price
    - future_price: predicted price after specified years
    """
    # Convert to DataFrame if needed
    if isinstance(house_features, dict):
        house_features = pd.DataFrame([house_features])
    elif isinstance(house_features, pd.Series):
        house_features = house_features.to_frame().T

    # Predict current price
    current_price = model.predict(house_features)[0]

    # Calculate future price
    future_price = current_price * ((1 + annual_growth) ** years)

    return {
        "current_price": current_price,
        "future_price": future_price,
        "years": years,
        "annual_growth_rate": annual_growth,
        "total_appreciation": ((future_price - current_price) / current_price) * 100
    }

# Test future price prediction
print("\n=== Testing Future Price Prediction ===")
sample_house = X_test.iloc[0]
print("Sample house features:")
print(sample_house)

for years in [1, 5, 10]:
    result = predict_future_price(sample_house, years)
    print(f"\n{years} year(s) projection:")
    print(f"  Current Price: ₱{result['current_price']:,.2f}")
    print(f"  Future Price: ₱{result['future_price']:,.2f}")
    print(f"  Total Appreciation: {result['total_appreciation']:.2f}%")

# ===== COMBINED SYSTEM: RECOMMEND + PREDICT =====
def recommend_with_price_projection(user_input, years=5, top_n=5, annual_growth=0.05):
    """
    Recommend homes and show price projections

    Parameters:
    - user_input: text description of desired property
    - years: years to project price
    - top_n: number of recommendations
    - annual_growth: expected annual growth rate
    """
    # Get recommendations
    recommendations = recommend_homes(user_input, top_n)

    results = []
    for idx, row in recommendations.iterrows():
        house_features = row[features]
        price_info = predict_future_price(house_features, years, annual_growth)

        results.append({
            "description": row["description"][:100] + "...",
            "location": row["location"],
            "bedrooms": row["bedrooms"],
            "bathrooms": row["bathrooms"],
            "current_price": price_info["current_price"],
            f"price_in_{years}_years": price_info["future_price"],
            "appreciation_%": price_info["total_appreciation"]
        })

    return pd.DataFrame(results)

# Test combined system
print("\n=== Testing Combined Recommendation + Price Projection ===")
results = recommend_with_price_projection(
    "3 bedroom house near school with parking",
    years=10,
    top_n=5
)
print(results)

# ===== SAVE MODELS =====
print("\n=== Saving Models ===")
pickle.dump(tfidf, open("tfidf_model.pkl", "wb"))
pickle.dump(tfidf_matrix, open("tfidf_matrix.pkl", "wb"))
pickle.dump(model, open("price_model.pkl", "wb"))
pickle.dump(df, open("housing_data.pkl", "wb"))  # Save the dataframe too

print("Models saved successfully!")

# If running in Google Colab, uncomment these lines:
from google.colab import files
files.download("tfidf_model.pkl")
files.download("tfidf_matrix.pkl")
files.download("price_model.pkl")
files.download("housing_data.pkl")

print("\n=== Training Complete ===")

FileNotFoundError: [Errno 2] No such file or directory: 'Housing Data Sets.csv'

In [2]:
import pandas as pd
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

print("=" * 60)
print("HOUSING RECOMMENDATION SYSTEM - MODEL TRAINING")
print("=" * 60)

# Load CSV
print("\n[1/6] Loading dataset...")
df = pd.read_csv("Housing Data Sets.csv")
print(f"✓ Loaded {len(df)} records")

# Print original columns to see what we're working with
print("\nOriginal columns:", df.columns.tolist())

# Rename columns to simpler names (adjust these based on your actual column names)
df.columns = [
    "description",
    "bathrooms",
    "bedrooms",
    "car_spaces",
    "floor_area_sqm",
    "land_size_sqm",
    "location",
    "price",
    "category"
]

# Fill missing values
print("\n[2/6] Cleaning data...")
df["description"] = df["description"].fillna("")
df["location"] = df["location"].fillna("")
df["category"] = df["category"].fillna("Unknown")

# Create combined text field for recommendation
df["text"] = df["description"] + " " + df["location"]

# Display first few rows
print("\nData preview:")
print(df.head())

# Check for missing values in numeric columns
print("\nMissing values before cleaning:")
print(df.isnull().sum())

# Fill missing numeric values with median
numeric_cols = ["bathrooms", "bedrooms", "car_spaces", "floor_area_sqm", "land_size_sqm", "price"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(df[col].median())

print("\nMissing values after cleaning:")
print(df.isnull().sum())
print("✓ Data cleaned successfully")

# ===== RECOMMENDATION SYSTEM =====
print("\n[3/6] Building Recommendation System...")
tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=5000
)
tfidf_matrix = tfidf.fit_transform(df["text"])
print(f"✓ TF-IDF matrix created: {tfidf_matrix.shape}")

def recommend_homes(user_input, top_n=5):
    """Recommend homes based on text description"""
    user_vec = tfidf.transform([user_input])
    similarity = cosine_similarity(user_vec, tfidf_matrix)
    top_indices = similarity[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

# Test recommendation
print("\nTesting recommendation system:")
print("Query: '3 bedroom house near school with parking'")
recommended = recommend_homes("3 bedroom house near school with parking", 5)
print("\nTop 5 Recommendations:")
print(recommended[["description", "bedrooms", "bathrooms", "price", "location"]].head())

# ===== PRICE PREDICTION MODEL =====
print("\n[4/6] Building Price Prediction Model...")
features = [
    "bedrooms",
    "bathrooms",
    "car_spaces",
    "floor_area_sqm",
    "land_size_sqm"
]

X = df[features]
y = df["price"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# Train model
print("\nTraining Random Forest model...")
model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)
print("✓ Model trained successfully")

# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Performance:")
print(f"  MAE: ₱{mae:,.2f}")
print(f"  R²: {r2:.4f}")

# ===== FUTURE PRICE PREDICTION =====
def predict_future_price(house_features, years, annual_growth=0.05):
    """
    Predict future price of a house

    Parameters:
    - house_features: dict or Series with keys: bedrooms, bathrooms, car_spaces, floor_area_sqm, land_size_sqm
    - years: number of years in the future
    - annual_growth: expected annual growth rate (default 5%)

    Returns:
    - dict with current_price, future_price, years, annual_growth_rate, total_appreciation
    """
    # Convert to DataFrame if needed
    if isinstance(house_features, dict):
        house_features = pd.DataFrame([house_features])
    elif isinstance(house_features, pd.Series):
        house_features = house_features.to_frame().T

    # Predict current price
    current_price = model.predict(house_features)[0]

    # Calculate future price
    future_price = current_price * ((1 + annual_growth) ** years)

    return {
        "current_price": current_price,
        "future_price": future_price,
        "years": years,
        "annual_growth_rate": annual_growth,
        "total_appreciation": ((future_price - current_price) / current_price) * 100
    }

# Test future price prediction
print("\n[5/6] Testing Future Price Prediction...")
sample_house = X_test.iloc[0]
print("\nSample house features:")
print(sample_house)

for years in [1, 5, 10]:
    result = predict_future_price(sample_house, years)
    print(f"\n{years} year(s) projection:")
    print(f"  Current Price: ₱{result['current_price']:,.2f}")
    print(f"  Future Price: ₱{result['future_price']:,.2f}")
    print(f"  Total Appreciation: {result['total_appreciation']:.2f}%")

# ===== COMBINED SYSTEM: RECOMMEND + PREDICT =====
def recommend_with_price_projection(user_input, years=5, top_n=5, annual_growth=0.05):
    """
    Recommend homes and show price projections

    Parameters:
    - user_input: text description of desired property
    - years: years to project price
    - top_n: number of recommendations
    - annual_growth: expected annual growth rate
    """
    # Get recommendations
    recommendations = recommend_homes(user_input, top_n)

    results = []
    for idx, row in recommendations.iterrows():
        house_features = row[features]
        price_info = predict_future_price(house_features, years, annual_growth)

        results.append({
            "description": row["description"][:100] + "..." if len(row["description"]) > 100 else row["description"],
            "location": row["location"],
            "bedrooms": row["bedrooms"],
            "bathrooms": row["bathrooms"],
            "current_price": price_info["current_price"],
            f"price_in_{years}_years": price_info["future_price"],
            "appreciation_%": price_info["total_appreciation"]
        })

    return pd.DataFrame(results)

# Test combined system
print("\nTesting Combined Recommendation + Price Projection...")
results = recommend_with_price_projection(
    "3 bedroom house near school with parking",
    years=10,
    top_n=5
)
print("\nCombined Results (Recommendations + 10-year Price Projection):")
print(results.to_string())

# ===== SAVE MODELS =====
print("\n[6/6] Saving Models...")

# Create models directory if it doesn't exist
os.makedirs("models", exist_ok=True)

# Save models
pickle.dump(tfidf, open("models/tfidf_model.pkl", "wb"))
print("✓ tfidf_model.pkl saved")

pickle.dump(tfidf_matrix, open("models/tfidf_matrix.pkl", "wb"))
print("✓ tfidf_matrix.pkl saved")

pickle.dump(model, open("models/price_model.pkl", "wb"))
print("✓ price_model.pkl saved")

pickle.dump(df, open("models/housing_data.pkl", "wb"))
print("✓ housing_data.pkl saved")



print("\n" + "=" * 60)
print("TRAINING COMPLETE!")
print("=" * 60)
print(f"Total properties: {len(df)}")
print(f"Unique categories: {df['category'].nunique()}")
print(f"Price range: ₱{df['price'].min():,.2f} - ₱{df['price'].max():,.2f}")
print("\nAll models saved to 'models/' directory")
print("You can now run: python app.py")
print("=" * 60)


HOUSING RECOMMENDATION SYSTEM - MODEL TRAINING


Saving Housing Data Sets.csv to Housing Data Sets.csv

✓ Loaded 7494 records from Housing Data Sets.csv
✓ TF-IDF matrix created: (7494, 5000)

Model Performance:
  MAE: ₱20,149,984.01
  R²: 0.0711

✓ All models saved in 'models/' folder
TRAINING COMPLETE!


In [1]:
import pandas as pd
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

print("=" * 60)
print("HOUSING RECOMMENDATION SYSTEM - MODEL TRAINING")
print("=" * 60)

# ===== 1️⃣ Upload and Load Dataset =====
from google.colab import files
uploaded = files.upload()  # prompts file upload

csv_filename = list(uploaded.keys())[0]
df = pd.read_csv(csv_filename)
print(f"\n✓ Loaded {len(df)} records from {csv_filename}")

# Rename columns
df.columns = [
    "description",
    "bathrooms",
    "bedrooms",
    "car_spaces",
    "floor_area_sqm",
    "land_size_sqm",
    "location",
    "price",
    "category"
]

# ===== 2️⃣ Data Cleaning =====
df["description"] = df["description"].fillna("")
df["location"] = df["location"].fillna("Not specified")  # default for missing locations
df["category"] = df["category"].fillna("Unknown")

# Fill numeric columns
numeric_cols = ["bathrooms", "bedrooms", "car_spaces", "floor_area_sqm", "land_size_sqm", "price"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(df[col].median())

# Create combined text for recommendation
df["text"] = df["description"] + " " + df["location"]

# ===== 3️⃣ Recommendation System (TF-IDF) =====
tfidf = TfidfVectorizer(stop_words="english", max_features=5000)
tfidf_matrix = tfidf.fit_transform(df["text"])
print(f"✓ TF-IDF matrix created: {tfidf_matrix.shape}")

def recommend_homes(user_input, top_n=5):
    user_vec = tfidf.transform([user_input])
    similarity = cosine_similarity(user_vec, tfidf_matrix)
    top_indices = similarity[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

# ===== 4️⃣ Price Prediction Model (Random Forest) =====
features = ["bedrooms", "bathrooms", "car_spaces", "floor_area_sqm", "land_size_sqm"]
X = df[features]
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(f"\nModel Performance:")
print(f"  MAE: ₱{mean_absolute_error(y_test, y_pred):,.2f}")
print(f"  R²: {r2_score(y_test, y_pred):.4f}")

# ===== 5️⃣ Future Price Prediction =====
def predict_future_price(house_features, years, annual_growth=0.05):
    if isinstance(house_features, dict):
        house_features = pd.DataFrame([house_features])
    elif isinstance(house_features, pd.Series):
        house_features = house_features.to_frame().T

    current_price = model.predict(house_features)[0]
    future_price = current_price * ((1 + annual_growth) ** years)
    return {
        "current_price": current_price,
        "future_price": future_price,
        "years": years,
        "annual_growth_rate": annual_growth,
        "total_appreciation": ((future_price - current_price) / current_price) * 100
    }

# ===== 6️⃣ Combined Recommendation + Price Projection =====
def recommend_with_price_projection(user_input, years=5, top_n=5, annual_growth=0.05):
    recommendations = recommend_homes(user_input, top_n)
    # Ensure required columns exist
    recommendations = recommendations[["description", "location"] + features]

    results = []
    for idx, row in recommendations.iterrows():
        loc = row["location"] if pd.notna(row["location"]) else "Not specified"
        house_features = row[features]
        price_info = predict_future_price(house_features, years, annual_growth)
        results.append({
            "description": row["description"][:100]+"..." if len(row["description"])>100 else row["description"],
            "location": loc,
            "bedrooms": row["bedrooms"],
            "bathrooms": row["bathrooms"],
            "current_price": price_info["current_price"],
            f"price_in_{years}_years": price_info["future_price"],
            "appreciation_%": price_info["total_appreciation"]
        })
    return pd.DataFrame(results)

# ===== 7️⃣ Save Models =====
os.makedirs("models", exist_ok=True)
pickle.dump(tfidf, open("models/tfidf_model.pkl", "wb"))
pickle.dump(tfidf_matrix, open("models/tfidf_matrix.pkl", "wb"))
pickle.dump(model, open("models/price_model.pkl", "wb"))
pickle.dump(df, open("models/housing_data.pkl", "wb"))
print("\n✓ All models saved in 'models/' folder")
print("TRAINING COMPLETE!")


HOUSING RECOMMENDATION SYSTEM - MODEL TRAINING


Saving Housing Data Sets.csv to Housing Data Sets.csv

✓ Loaded 7494 records from Housing Data Sets.csv
✓ TF-IDF matrix created: (7494, 5000)

Model Performance:
  MAE: ₱20,149,984.01
  R²: 0.0711

✓ All models saved in 'models/' folder
TRAINING COMPLETE!


In [4]:
!pip install scikit-learn==1.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 112.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
